<a href="https://colab.research.google.com/github/tcmb1987/EE274-lossy-text-compression/blob/main/stopword_pruner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Approach 1: Prune stopwords first, then embed with some word2vec <-> sentence2vec adaptation

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
stop_words = set(stopwords.words('english'))
  
def prune_sentence(sentence, stop_words):
  word_tokens = word_tokenize(sentence)
  return ' '.join([w for w in word_tokens if not w.lower() in stop_words and not w.lower() in string.punctuation]) 

In [ ]:
from sent2vec.vectorizer import Vectorizer

sentences = [
    "In the beginning, God created the heavens and the earth.",
    "And the earth was without form, and void, and darkness lay upon the face of the deep.",
    "And the spirit of God looked upon the face of the waters.",
    "The yellow dog sits.",
    "The tall woman stands.",
    "The ground beef tonight was thoroughly satisfying.",
    "I love to eat fried eggs.",
    "My friend wants to marry me.",
    "I know it is wet and the sun is not sunny, but we can have lots of good fun that is funny.",
    "I know wet sun not sunny, we can lots good fun is funny",
    "I know that it is wet and the sun is not sunny, however we can have lots of good fun if it is funny."
]

pruned_sentences = [prune_sentence(sentence, stop_words) for sentence in sentences]

pruned_sentences

['beginning God created heavens earth',
 'earth without form void darkness lay upon face deep',
 'spirit God looked upon face waters',
 'yellow dog sits',
 'tall woman stands',
 'ground beef tonight thoroughly satisfying',
 'love eat fried eggs',
 'friend wants marry',
 'know wet sun sunny lots good fun funny',
 'know wet sun sunny lots good fun funny',
 'know wet sun sunny however lots good fun funny']

In [ ]:
vectorizer = Vectorizer()
vectorizer.run(pruned_sentences)
vectors = vectorizer.vectors

Initializing Bert distilbert-base-uncased
Vectorization done on cpu


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# How does this thing vectorize them to all have the same dimensions? So curious.
# Since it does, though, we can probably just use a nice supervised ML model right here to see what's up.

pruned_sentence_vectors = vectors

In [ ]:
original_vectorizer = Vectorizer()
original_vectorizer.run(sentences)
original_sentence_vectors = original_vectorizer.vectors

Initializing Bert distilbert-base-uncased
Vectorization done on cpu


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Initialize some sentences to determine distortion metrics (cosine distance) between original sentence with its cut version and a potential reconstructed one.

Get distance and print results.

In [ ]:
from scipy.spatial import distance

dist_cut = distance.cosine(vectors[0], vectors[1])
print("Distance between original sentence and cut sentence is", dist_cut)
dist_rec = distance.cosine(vectors[0], vectors[2])
print("Distance between original sentence and reconstructed sentence is", dist_rec)

Distance between original sentence and cut sentence is 0.22216004133224487
Distance between original sentence and reconstructed sentence is 0.023939311504364014
